In [ ]:
import pytesseract
from PIL import Image
import cv2
import re

# If necessary, specify the Tesseract executable path
# For Windows, it might look like this:
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load the image
image_path = 'path_to_your_image/image.png'
image = cv2.imread(image_path)

# Preprocess the image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
gray = cv2.medianBlur(gray, 3)

# Use Tesseract to extract text
text = pytesseract.image_to_string(gray)

# Print the extracted text
print(text)

# Extract relevant information using regular expressions
def extract_info(text):
    data = {}

    # Extract ABN
    abn_match = re.search(r'ABN:\s*([\d\s]+)', text)
    data['ABN'] = abn_match.group(1).strip() if abn_match else None

    # Extract Company Name and Address
    company_match = re.search(r'Store Name:\s*(.*)', text)
    data['Company Name'] = company_match.group(1).strip() if company_match else None

    address_match = re.search(r'Address:\s*(.*)', text)
    data['Address'] = address_match.group(1).strip() if address_match else None

    # Extract GST
    gst_match = re.search(r'GST Included\s*\$(\d+\.\d+)', text)
    data['GST'] = gst_match.group(1).strip() if gst_match else None

    # Extract Total Cost
    total_cost_match = re.search(r'Your Total\s*\$(\d+\.\d+)', text)
    data['Total Cost'] = total_cost_match.group(1).strip() if total_cost_match else None

    # Extract Item List
    item_list = []
    item_pattern = re.compile(r'(.+?)\s+\$\d+\.\d+\s+\d+\s+\$\d+\.\d+')
    for match in item_pattern.finditer(text):
        item_list.append(match.group(0))

    data['Items'] = item_list

    return data

# Extracted Information
extracted_info = extract_info(text)
print(extracted_info)
